# Flower Classification

This model classifies two type of flowers -  Rose and Sunflower. You can add  classes or  make changes in the model as per your requirement. 

## Import required libraries

In [1]:
import os, cv2, math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
from tqdm import tqdm
from shutil import copyfile
%matplotlib inline

In [2]:
#Path for the dataset
DATASET_PATH = '/content/gdrive/My Drive/FlowerDataset'

In [ ]:
# Splitting the data into train and valid data

train_set = {}
valid_set = {}

for path in os.listdir(DATASET_PATH):
  path_dir = os.path.join(DATASET_PATH, path) # e.g. DATASET_PATH/'0'
  path_files = os.listdir(path_dir)
  # Training set's size is 80% of the data
  train_list , test_list = train_test_split(path_files, test_size = 0.2)
  
  train_set[path] = train_list
  valid_set[path] = validation_list

## For Training data

In [ ]:
for path in tqdm(train_set.keys()):
  path_dir = os.path.join(DATASET_PATH, 'training_set', 'class_0' + str(path))
  os.makedirs(path_dir)
  for file in train_set[path]:
    # src path is DATASET_PATH/'0'/file
    src = os.path.join(DATASET_PATH, path, file)
    # dest path is DATASET_PATH/'training_set'/'class_00'
    # to accomodate for the directory format required by flow_from_directory method in keras
    dest = os.path.join(path_dir, file)
    copyfile(src, dest)

100%|██████████| 2/2 [00:59<00:00, 29.74s/it]


## For validation Data

In [ ]:
for path in tqdm(valid_set.keys()):
  path_dir = os.path.join(DATASET_PATH, 'validation_set', 'class_0' + str(path))
  os.makedirs(path_dir)
  for file in valid_set[path]:
    # src path is DATASET_PATH/'0'/file
    src = os.path.join(DATASET_PATH, path, file)
    # dest path is DATASET_PATH/'training_set'/'class_00'
    # to accomodate for the directory format required by flow_from_directory method in keras
    dest = os.path.join(path_dir, file)
    copyfile(src, dest)

100%|██████████| 2/2 [00:13<00:00,  6.61s/it]


## Data Augmentation

In [21]:
#For training data
train_datagen = ImageDataGenerator(rescale = 1.0/255.0, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

#For validation data, we only rescale the pictures
valid_datagen = ImageDataGenerator(rescale = 1.0/255.0)


train_batches = train_datagen.flow_from_directory(os.path.join(DATASET_PATH, 'training_set'), target_size = (64, 64), batch_size = 32, shuffle = True)
valid_batches = valid_datagen.flow_from_directory(os.path.join(DATASET_PATH, 'validation_set'), target_size = (64, 64), batch_size = 32, shuffle = True)


Found 345 images belonging to 2 classes.
Found 74 images belonging to 2 classes.


## Build and Train the model

In [10]:
def create_model():

    model = Sequential()
    
    #Input Layer
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(64,64,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    
    #Adding convolutional layer
    Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.3),
    
    #Flattening before the fully-connected network
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    #Final Output Layer
    model.add(Dense(2, activation='softmax'))
  
   #Compile the model
    model.compile(loss= 'categorical_crossentropy',
         optimizer=tf.keras.optimizers.Adam(),
         metrics=['accuracy'])
      
    return model

model = create_model()


In [11]:
model.fit_generator(train_batches, 
                    steps_per_epoch = math.ceil(train_batches.n / train_batches.batch_size),
                    validation_data = valid_batches,
                    validation_steps = math.ceil(valid_batches.n / valid_batches.batch_size), epochs = 30)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
11/11 [==============================] - 3s 300ms/step - loss: 0.7043 - accuracy: 0.5681 - val_loss: 0.5364 - val_accuracy: 0.8243
Epoch 2/30
11/11 [==============================] - 3s 295ms/step - loss: 0.5343 - accuracy: 0.7594 - val_loss: 0.4213 - val_accuracy: 0.8378
Epoch 3/30
11/11 [==============================] - 3s 284ms/step - loss: 0.4218 - accuracy: 0.8377 - val_loss: 0.3351 - val_accuracy: 0.8784
Epoch 4/30
11/11 [==============================] - 3s 284ms/step - loss: 0.3663 - accuracy: 0.8435 - val_loss: 0.2837 - val_accuracy: 0.8919
Epoch 5/30
11/11 [==============================] - 3s 290ms/step - loss: 0.3549 - accuracy: 0.8609 - val_loss: 0.2328 - val_accuracy: 0.9189
Epoch 6/30
11/11 [==============================] - 3s 289ms/step - loss: 0.2916 - accuracy: 0.8870 - val_loss: 0.2435 - val_accuracy: 0.8919
Epoch 7/30
11/11 [==============================] - 3s 285ms/step - loss

## Saving Convolutional model

In [19]:
tensorflow_keras_file = "flower.h5"
tf.keras.models.save_model(model, tensorflow_keras_file)



## Converting .h5 model  into .tflite file

In [20]:
model = tf.keras.models.load_model("flower.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("flower.tflite", "wb").write(tflite_model)

16784220